# Ingmar05/Dreambooth-Stable-Diffusion
https://github.com/Ingmar05/Dreambooth-Stable-Diffusion

## Settings

In [ ]:
# Mandatory Settings
project_name = "order1"

#Optional Settings
dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
token = "firstNameLastName" #This is the unique token you are incorporating into the stable diffusion model.
class_word = "person" # Match class_word to the category of the regularization images you chose above. Typical uses are "man", "person", "woman"
max_training_steps = 2000 # MAX STEPS # How many steps do you want to train for?

print("✅ Settings Applied")



## Training

In [ ]:
# BUILD ENV
!pip install numpy==1.23.1
!pip install pytorch-lightning==1.7.6
!pip install csv-logger
!pip install torchmetrics==0.11.1
!pip install torch-fidelity==0.3.0
!pip install albumentations==1.1.0
!pip install opencv-python==4.7.0.72
!pip install pudb==2019.2
!pip install omegaconf==2.1.1
!pip install pillow==9.4.0
!pip install einops==0.4.1
!pip install transformers==4.25.1
!pip install kornia==0.6.7
!pip install diffusers[training]==0.3.0
!pip install captionizer==1.0.1
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .
!pip install huggingface_hub
!pip install gitpython

# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

#Download Regularization Images
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# Training

# If you are training a person's face, set this to True
i_am_training_a_persons_face = False

flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

# 0 Saves the checkpoint when max_training_steps is reached.
# 250 saves the checkpoint every 250 steps as well as when max_training_steps is reached.
save_every_x_steps = 0

reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --project_name "{project_name}" \
 --debug False \
 --max_training_steps {max_training_steps} \
 --token "{token}" \
 --training_model "model.ckpt" \
 --training_images "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --regularization_images "{reg_data_root}" \
 --class_word "{class_word}" \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}

## Wait for Training to Finish and Download Model

Download the trained model from */workspace/Dreambooth-Stable-Diffusion/trained_models/*